In [2]:
import tensorflow as tf
import os
import random
import numpy as np

def seed_everything(seed):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)

SEED = 22
seed_everything(seed=SEED)

In [2]:
from functools import reduce
from itertools import accumulate

landmark_lens = (
    (33, 4),
    (468, 3),
    (21, 3),
    (21, 3)
)
landmark_locs = list(accumulate(landmark_lens, lambda a, b: a + b[0]*b[1], initial=0))
landmarks_len = reduce(lambda r, loc: r + loc[0] * loc[1], landmark_lens, 0)
print(landmark_locs)

[0, 132, 1536, 1599, 1662]


In [3]:
labels = [label for label in os.listdir('tracks_binary') if os.path.isdir(f'tracks_binary/{label}')]
NUM_CLASSES = len(labels)

labels_tensor = tf.constant(labels)
ids_tensor = tf.constant(range(len(labels)))

ids_from_labels = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(
        labels_tensor,
        ids_tensor
    ),
    default_value=-1
)

labels_from_ids = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(
        ids_tensor,
        labels_tensor
    ),
    default_value=""
)

def to_categorical(label):
    return tf.one_hot(
        ids_from_labels.lookup(label),
        depth=NUM_CLASSES
    )

In [2]:
import tensorflow as tf
# Create Tensor
tensor1 = tf.range(5)

#Create dataset, this will return object of TensorSliceDataset
dataset = tf.data.Dataset.from_tensor_slices(tensor1)

In [3]:
print("dataset after applying batch and repeat")
dataset = dataset.repeat(6).batch(batch_size=2)
for i in dataset:
    print(i)

dataset after applying batch and repeat
tf.Tensor([0 1], shape=(2,), dtype=int32)
tf.Tensor([2 3], shape=(2,), dtype=int32)
tf.Tensor([4 0], shape=(2,), dtype=int32)
tf.Tensor([1 2], shape=(2,), dtype=int32)
tf.Tensor([3 4], shape=(2,), dtype=int32)
tf.Tensor([0 1], shape=(2,), dtype=int32)
tf.Tensor([2 3], shape=(2,), dtype=int32)
tf.Tensor([4 0], shape=(2,), dtype=int32)
tf.Tensor([1 2], shape=(2,), dtype=int32)
tf.Tensor([3 4], shape=(2,), dtype=int32)
tf.Tensor([0 1], shape=(2,), dtype=int32)
tf.Tensor([2 3], shape=(2,), dtype=int32)
tf.Tensor([4 0], shape=(2,), dtype=int32)
tf.Tensor([1 2], shape=(2,), dtype=int32)
tf.Tensor([3 4], shape=(2,), dtype=int32)


In [72]:
a = tf.reshape(tf.range(0.0, 16.0), [-1, 2, 4])
b = tf.reshape(tf.range(10.0, 26.0), [-1, 2, 4])
c = tf.reshape(tf.range(20.0, 36.0), [-1, 2, 4])
d = tf.reshape(tf.range(30.0, 46.0), [-1, 2, 4])

In [73]:
a

<tf.Tensor: shape=(2, 2, 4), dtype=float32, numpy=
array([[[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.]],

       [[ 8.,  9., 10., 11.],
        [12., 13., 14., 15.]]], dtype=float32)>

In [74]:
d

<tf.Tensor: shape=(2, 2, 4), dtype=float32, numpy=
array([[[30., 31., 32., 33.],
        [34., 35., 36., 37.]],

       [[38., 39., 40., 41.],
        [42., 43., 44., 45.]]], dtype=float32)>

In [83]:
from tensorflow import reduce_max, reduce_min
def calc_bounding(pose, face, lh, rh):
  max_x = max(reduce_max(pose[:, :, :1]), reduce_max(face[:, :, :1]), reduce_max(lh[:, :, :1]), reduce_max(rh[:, :, :1]))
  min_x = min(reduce_min(pose[:, :, :1]), reduce_min(face[:, :, :1]), reduce_min(lh[:, :, :1]), reduce_min(rh[:, :, :1]))

  max_y = max(reduce_max(pose[:, :, 1:2]), reduce_max(face[:, :, 1:2]), reduce_max(lh[:, :, 1:2]), reduce_max(rh[:, :, 1:2]))
  min_y = min(reduce_min(pose[:, :, 1:2]), reduce_min(face[:, :, 1:2]), reduce_min(lh[:, :, 1:2]), reduce_min(rh[:, :, 1:2]))

  diff = (max_x - min_x, max_y - min_y)
  mid = ((max_x + min_x)/2, (max_y + min_y)/2)
  return (diff, mid)

def scale(scale, pose, face, lh , rh):
  scale = tf.cast(scale, dtype=tf.float32)
  diff, mid = calc_bounding(pose, face, lh, rh)
  pose_shape, face_shape, lh_shape, rh_shape = tf.shape(pose), tf.shape(face), tf.shape(lh), tf.shape(rh)
  pose_add = tf.tile([[[mid[0], mid[1], 0, 0]]], [pose_shape[0], pose_shape[1], 1])
  face_add = tf.tile([[[mid[0], mid[1], 0, 0]]], [face_shape[0], face_shape[1], 1])
  lh_add = tf.tile([[[mid[0], mid[1], 0, 0]]], [lh_shape[0], lh_shape[1], 1])
  rh_add = tf.tile([[[mid[0], mid[1], 0, 0]]], [rh_shape[0], rh_shape[1], 1])
  
  pose_scale = tf.tile([[[scale, scale, 1, 1]]], [pose_shape[0], pose_shape[1], 1])
  face_scale = tf.tile([[[scale, scale, 1, 1]]], [face_shape[0], face_shape[1], 1])
  lh_scale = tf.tile([[[scale, scale, 1, 1]]], [lh_shape[0], lh_shape[1], 1])
  rh_scale = tf.tile([[[scale, scale, 1, 1]]], [rh_shape[0], rh_shape[1], 1])

  scaled_pose = (pose + pose_add) * pose_scale
  scaled_face = (face + face_add) * face_scale
  scaled_lh = (lh + lh_add) * lh_scale
  scaled_rh = (rh + rh_add) * rh_scale

  return (scaled_pose, scaled_face, scaled_lh, scaled_rh)

scale(0.1, a, b, c, d)

(<tf.Tensor: shape=(2, 2, 4), dtype=float32, numpy=
 array([[[ 2.1000001,  2.3      ,  2.       ,  3.       ],
         [ 2.5      ,  2.7      ,  6.       ,  7.       ]],
 
        [[ 2.9      ,  3.1000001, 10.       , 11.       ],
         [ 3.3      ,  3.5      , 14.       , 15.       ]]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2, 4), dtype=float32, numpy=
 array([[[ 3.1000001,  3.3      , 12.       , 13.       ],
         [ 3.5      ,  3.7      , 16.       , 17.       ]],
 
        [[ 3.9      ,  4.1      , 20.       , 21.       ],
         [ 4.3      ,  4.5      , 24.       , 25.       ]]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2, 4), dtype=float32, numpy=
 array([[[ 4.1      ,  4.3      , 22.       , 23.       ],
         [ 4.5      ,  4.7000003, 26.       , 27.       ]],
 
        [[ 4.9      ,  5.1      , 30.       , 31.       ],
         [ 5.3      ,  5.5      , 34.       , 35.       ]]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2, 4), dtype=float32, numpy=
 array([[[ 5.1    

In [62]:
mid = 6
scale = 0.5
z = tf.cast(a[:, :, :1] + mid, dtype=tf.float32) * scale
z

<tf.Tensor: shape=(2, 2, 1), dtype=float32, numpy=
array([[[3.],
        [5.]],

       [[7.],
        [9.]]], dtype=float32)>

In [69]:
mid = (6, 8)
s = tf.shape(a)
add = tf.tile([[[mid[0], mid[1], 0, 0]]], [s[0], s[1], 1])
scl = tf.tile([[[scale, scale, 0, 0]]], [s[0], s[1], 1])
(tf.cast(a, dtype=tf.float32) + tf.cast(add, dtype=tf.float32)) * scl

<tf.Tensor: shape=(2, 2, 4), dtype=float32, numpy=
array([[[ 3. ,  4.5,  0. ,  0. ],
        [ 5. ,  6.5,  0. ,  0. ]],

       [[ 7. ,  8.5,  0. ,  0. ],
        [ 9. , 10.5,  0. ,  0. ]]], dtype=float32)>

In [60]:
a

<tf.Tensor: shape=(2, 2, 4), dtype=int32, numpy=
array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7]],

       [[ 8,  9, 10, 11],
        [12, 13, 14, 15]]])>

In [42]:
d[:, :, 1:2]

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[31],
        [35]],

       [[39],
        [43]]])>

In [35]:
max(reduce_max(a[:, :, :1]), 11)

<tf.Tensor: shape=(), dtype=int32, numpy=12>

In [4]:
def get_ds_split(ds, ds_size, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=1000):
  assert (train_split + test_split + val_split) == 1
  
  if shuffle:
    ds = ds.shuffle(shuffle_size, seed=SEED)
  
  train_size = int(train_split * ds_size)
  val_size = int(val_split * ds_size)
  
  train_ds = ds.take(train_size)
  val_ds = ds.skip(train_size).take(val_size)
  test_ds = ds.skip(train_size).skip(val_size)
  
  return train_ds, val_ds, test_ds

In [5]:
train_ds, val_ds, test_ds = get_ds_split(dataset, len(dataset), shuffle=False)


In [6]:
for i in train_ds:
  print(i)

tf.Tensor([0 1], shape=(2,), dtype=int32)
tf.Tensor([2 3], shape=(2,), dtype=int32)
tf.Tensor([4 0], shape=(2,), dtype=int32)
tf.Tensor([1 2], shape=(2,), dtype=int32)
tf.Tensor([3 4], shape=(2,), dtype=int32)
tf.Tensor([0 1], shape=(2,), dtype=int32)
tf.Tensor([2 3], shape=(2,), dtype=int32)
tf.Tensor([4 0], shape=(2,), dtype=int32)
tf.Tensor([1 2], shape=(2,), dtype=int32)
tf.Tensor([3 4], shape=(2,), dtype=int32)
tf.Tensor([0 1], shape=(2,), dtype=int32)
tf.Tensor([2 3], shape=(2,), dtype=int32)


In [37]:
for i in val_ds:
  print(i)

tf.Tensor([4 0], shape=(2,), dtype=int32)


In [38]:
for i in test_ds:
  print(i)

tf.Tensor([1 2], shape=(2,), dtype=int32)
tf.Tensor([3 4], shape=(2,), dtype=int32)


In [20]:
def process_binary(file_path):
    label = tf.strings.split(file_path, os.sep)[-2]

    raw = tf.io.read_file(file_path)
    data = tf.io.decode_raw(raw, tf.float32)
    data = tf.reshape(data, [-1, landmarks_len])

    pose = tf.reshape(data[:, 0:132], [-1, 33, 4])
    face = tf.reshape(data[:, 132:1536], [-1, 468, 3])
    lh = tf.reshape(data[:, 1536:1599], [-1, 21, 3])
    rh = tf.reshape(data[:, 1599:1662], [-1, 21, 3])

    return (pose, face, lh, rh), to_categorical(label)

In [21]:
FRAMES = 64

def flatten(x):
    pose = tf.reshape(x[0], shape=[-1, 132])
    face = tf.reshape(x[1], shape=[-1, 1404])
    lh = tf.reshape(x[2], shape=[-1, 63])
    rh = tf.reshape(x[3], shape=[-1, 63])

    return tf.concat([pose, face, lh, rh], axis=1)


def random_window(x):
    def pad(x):
        missing = FRAMES - size
        start_pad = tf.math.ceil(missing / 2)
        end_pad = tf.math.floor(missing / 2)
        return tf.concat([
            tf.tile([x[0]], [start_pad, 1]),
            x,
            tf.tile([x[-1]], [end_pad, 1])
        ], axis=0)

    def random_slice(x):
        i = tf.random.uniform(shape=(), maxval=size+1-FRAMES, dtype=tf.int32)
        return x[i: i+FRAMES]

    size = tf.shape(x)[0]
    return tf.cond(
        size < FRAMES,
        lambda: pad(x),
        lambda: random_slice(x)
    )
    

def prepare(ds):
    ds = ds.map(lambda x, y: (flatten(x), y), num_parallel_calls=tf.data.AUTOTUNE)

    ds = ds.map(lambda x, y: (random_window(x), y), num_parallel_calls=tf.data.AUTOTUNE)

    ds = ds.shuffle(1000, seed=SEED)

    ds = ds.batch(32)

    return ds.prefetch(buffer_size=tf.data.AUTOTUNE)

In [22]:
import math

ds = tf.data.Dataset.list_files('tracks_binary/*/*')
ds = ds.map(process_binary)

ds = ds.shuffle(1000, seed=SEED)

dataset_size = len(ds)
train_size = math.floor(0.8 * dataset_size)

train_ds = ds.take(train_size)
# train_ds = prepare(train_ds)

val_ds = ds.skip(train_size)
# val_ds = prepare(val_ds)

# ds = prepare(ds)

In [24]:
ds

<ShuffleDataset shapes: (((None, 33, 4), (None, 468, 3), (None, 21, 3), (None, 21, 3)), (10,)), types: ((tf.float32, tf.float32, tf.float32, tf.float32), tf.float32)>

In [23]:
print(len(train_ds), len(val_ds))

255 64


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorflow import keras

In [9]:
TRIAL = 1
log_dir = os.path.join('Heh/{}'.format(TRIAL))
tb_callback = TensorBoard(log_dir=log_dir)
es_callback = EarlyStopping(monitor='val_loss', patience=20)
lr_callback = ReduceLROnPlateau(monitor='val_loss', patience=25)

In [10]:
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(FRAMES, 1662)))
model.add(Dense(64, activation='relu'))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dense(64, activation='relu'))
model.add(Bidirectional(LSTM(64, return_sequences=False, dropout=0.2)))
model.add(Dense(NUM_CLASSES, activation='softmax'))

In [11]:
opt = keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 64, 128)          884224    
 l)                                                              
                                                                 
 dense (Dense)               (None, 64, 64)            8256      
                                                                 
 bidirectional_1 (Bidirectio  (None, 64, 128)          66048     
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 64, 64)            8256      
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              66048     
 nal)                                                            
                                                        

In [13]:
history = model.fit(train_ds, validation_data=val_ds, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
8/8 [==============================] - 17s 614ms/step - loss: 2.2939 - accuracy: 0.0784 - val_loss: 2.2457 - val_accuracy: 0.1406
Epoch 2/2000
8/8 [==============================] - 2s 202ms/step - loss: 2.2519 - accuracy: 0.1176 - val_loss: 2.2501 - val_accuracy: 0.1406
Epoch 3/2000
8/8 [==============================] - 2s 197ms/step - loss: 2.2309 - accuracy: 0.1490 - val_loss: 2.2304 - val_accuracy: 0.1875
Epoch 4/2000
8/8 [==============================] - 2s 224ms/step - loss: 2.2160 - accuracy: 0.1490 - val_loss: 2.2143 - val_accuracy: 0.0938
Epoch 5/2000
8/8 [==============================] - 2s 227ms/step - loss: 2.2125 - accuracy: 0.1490 - val_loss: 2.1384 - val_accuracy: 0.2188
Epoch 6/2000
8/8 [==============================] - 2s 197ms/step - loss: 2.1834 - accuracy: 0.1529 - val_loss: 2.1715 - val_accuracy: 0.2344
Epoch 7/2000
8/8 [==============================] - 2s 245ms/step - loss: 2.1802 - accuracy: 0.1569 - val_loss: 2.1495 - val_accuracy: 0.2031
Epoch

KeyboardInterrupt: 

In [15]:
model.save('steven{}.h5'.format(TRIAL+1))

In [16]:
print(len(train_ds), len(val_ds))

8 2
